In [ ]:
import glob
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from model import data_analysis
from plotly.subplots import make_subplots

DATA_PATH = '../data/raw/'

In [ ]:
FILENAME = 'raw_NIST1001_20200715_024114_GMT.bin.marked.csv.csv'
# 'raw_NIST1052_20200716_193850_GMT.bin.marked.csv.csv'
# 'raw_NIST1051_20200716_192656_GMT.bin.marked.csv.csv'
# 'raw_NIST1021_20200716_213906_GMT.bin.marked.csv'
# # this data process above assumes a simple <60 dBm filter to set false positive
# 'raw_NIST1014_20200708_014055_GMT.bin.marked.csv.csv'
# 'raw_NIST1001_20200715_024114_GMT.bin.marked.csv.csv'

df = data_analysis.load_marked(os.path.join(DATA_PATH, FILENAME))
df.head()

In [ ]:
df['close encounter'][1]

In [ ]:
figure = data_analysis.plot_rssi(df['time'], df['rssi'])
# add axes labels
plt.show()

In [ ]:
[unique_ids,unique_counts] = data_analysis.get_unique(df)
unique_ids

In [ ]:
figure = data_analysis.plot_data_check(df)
figure.show()

In [ ]:
WINDOW_SIZE = 6
REQD_READINGS = 4
CLOSE_LEVEL = -50
# default value is -60

fig = make_subplots(rows=1, cols=len(unique_ids[unique_counts>1])) #, shared_yaxes=True
ylim0 = 1.05*min(df['rssi'])
ylim1 = max(df['rssi'])
fig.update_xaxes(title_text = "Time")
fig.update_yaxes(title_text = "RSSI", row=1, col=1)
fig.update_yaxes(range=[ylim0, ylim1])
figcolors = px.colors.qualitative.Plotly

for eid in range(len(unique_ids[unique_counts>1])):
    df_eid = df[df['encounter_id'] == unique_ids[unique_counts>1][eid]].copy()
    df_eid = data_analysis.mofn_filter(df_eid, WINDOW_SIZE, REQD_READINGS, CLOSE_LEVEL)
    df_eid.set_index('time', inplace=True)
    df_eid['comment_index'] = pd.notna(df_eid['close encounter'])
    
    transitions = data_analysis.transitions(df_eid)
   
#     print(transitions)

    plot_index = eid+1
    
    for i in range (0, len(transitions), 2):
        fig.add_trace(go.Scatter(x=[transitions[i], transitions[i], transitions[i+1], transitions[i+1], transitions[i]], 
                                 y=[ylim0, ylim1, ylim1, ylim0, ylim0], mode="lines",
                                 fill="toself",line_width=0, fillcolor=figcolors[eid], opacity = 0.5, showlegend=False),
                     row=1, col=plot_index)
#         or use fillcolor = 'MediumPurple'
    
    fig.add_trace(go.Scatter(x=df_eid.index, y=df_eid.rssi,
                             mode="markers", marker_color=figcolors[eid], 
                             name = df_eid.encounter_id[1][:4], text=df_eid["close encounter"]), 
                  row=1, col=plot_index)


    del df_eid

fig.show()

# TODO change marker type based on 'close encounter' text
# e.g. large circle if comment other than NaN, NO, YES, FALSE POSITIVE
# e.g. triangle if comment NO, FALSE POSITIVE
# e.g. current marker if NaN, YES